In [ ]:
import pandas as pd
import geopandas as gpd
import random
import numpy as np
import seaborn as sns
from aves.models.datafusion import DataFusionModel
from aves.data import census, eod
from aves.features.utils import standardize_columns, normalize_rows
from aves.models.grid import S2Grid
from aves.visualization.figures import small_multiples_from_geodataframe
from aves.visualization.maps import choropleth_map
from aves.visualization.colors import add_ranged_color_legend, color_legend
from seaborn import color_palette
from gensim.utils import deaccent
from aves.features.geo import to_point_geodataframe

In [ ]:
from pathlib import Path
AVES_ROOT = Path("../..")

## Datos Electorales

In [ ]:
turnout = pd.read_csv(AVES_ROOT / 'data' / 'external' / 'servel_2021' / 'votos_comuna_presidenciales_1ra_2021.csv', index_col=0)
turnout

In [ ]:
geo_turnout = pd.read_csv(AVES_ROOT / 'data' / 'external' / 'servel_2021' / 'resultados_presidenciales_1ra_2021.csv', dtype={'s2_cellid': 'str'})
geo_turnout

In [ ]:

zones = gpd.read_file(AVES_ROOT / "data" / "processed" / "scl_zonas_urbanas.json").set_index('ID')
zones.plot()

In [ ]:

comunas = census.read_census_map('comuna', path=AVES_ROOT / "data" / "external" / "censo_2017_R13").to_crs(zones.crs)

In [ ]:


square_grid = S2Grid.from_geodf(zones, grid_level=13).geodf
square_grid

In [ ]:
relevant_cell_ids = list(gpd.sjoin(square_grid, zones, op='intersects')['s2_cellid'].unique())
len(relevant_cell_ids), len(square_grid)

In [ ]:
square_grid[square_grid['s2_cellid'].isin(relevant_cell_ids)].plot()

In [ ]:
grid_with_data = square_grid.merge(geo_turnout, how='left').set_index('s2_cellid')
grid_with_data

In [ ]:
grid_with_data.describe()

In [ ]:
bins = [0.0, 0.20, 0.40, 0.50, 0.60]

In [ ]:


fig, axes = small_multiples_from_geodataframe(grid_with_data, 2)

palette = color_palette('magma', n_colors=4)

for ax, col in zip(axes, ["GABRIEL BORIC FONT", "JOSE ANTONIO KAST RIST"]):
    choropleth_map(
        ax,
        grid_with_data,
        col,
        binning='custom',
        bins=bins, legend=None, palette=palette
    )
    comunas.plot(ax=ax, facecolor="none", edgecolor="#abacab", linewidth=1)
    ax.set_title(col.title())

fig.tight_layout()
legend_ax = fig.add_axes([0.35, -0.03, 0.3, 0.02])
legend_ax.set_title('Proporción de los votos', fontsize='small', loc='left')
color_legend(legend_ax, palette, bins=bins)
fig.tight_layout()


## Datos de Transporte

In [ ]:


data_eod = eod.read_trips().merge(eod.read_homes()).merge(eod.read_people())
data_eod.head()

In [ ]:
recurrent_trips = data_eod[
    (data_eod["Proposito"].isin(["volver a casa"]))
    & (data_eod["DistEuclidiana"] >= 500)
    & (data_eod["TiempoViaje"] <= 120)
    & (pd.notnull(data_eod["SectorDestino"]))
].copy()
recurrent_trips.head()

In [ ]:
recurrent_trips["bin_tiempo_de_viaje"] = pd.cut(recurrent_trips["TiempoViaje"], bins=6)
recurrent_trips["bin_tiempo_de_viaje"].value_counts()

In [ ]:
recurrent_trips["PesoLaboral"] = (
    recurrent_trips["FactorLaboralNormal"] * recurrent_trips["Factor_LaboralNormal"]
)

In [ ]:


comuna_x_transporte = (
    recurrent_trips.groupby(["Comuna", "bin_tiempo_de_viaje"])["PesoLaboral"]
    .sum()
    .unstack()
    .pipe(normalize_rows)
    .reset_index()
    .assign(Comuna=lambda x: x["Comuna"].map(deaccent))
    .set_index("Comuna")
    #.pipe(lambda x: ensure_index(x, turnout))
    #.loc[geoloc_comunas_index]
)

comuna_x_transporte

## Datos que caracterizan la población

También usaremos la encuesta Casen 2017. La pueden descargar aquí:

- [Base de datos](http://observatorio.ministeriodesarrollosocial.gob.cl/storage/docs/casen/2017/casen_2017_stata.rar)
- [Libro de códigos](http://observatorio.ministeriodesarrollosocial.gob.cl/storage/docs/casen/2017/Libro_de_Codigos_Casen_2017.pdf)

No olviden guardarla en la carpeta correspondiente (`/data/external/casen_2017`).

In [ ]:
casen = pd.read_stata(AVES_ROOT / 'data' / 'external' / 'casen_2017' / 'Casen 2017.dta', convert_categoricals=False)
casen.head()

In [ ]:
codes = pd.read_excel(AVES_ROOT / 'data' / 'external' / 'casen_2017' / 'CUT_2018_v04.xls')
codes


In [ ]:
casen_comunas = casen.join(
    codes.set_index("Código Comuna 2018")["Nombre Comuna"]
    .str.upper()
    .map(deaccent)
    .rename("COMUNA"),
    on="comuna",
).pipe(lambda x: x[x["COMUNA"].isin(comuna_x_transporte.index)])
casen_comunas


In [ ]:
comunas_x_ingreso = (
    casen_comunas.groupby(["COMUNA", "dautr"])["expr"]
    .sum()
    .unstack(fill_value=0)
)
comunas_x_ingreso

In [ ]:
comunas_x_drogas = (
    casen_comunas.groupby(["COMUNA", "v38b"])["expr"]
    .sum()
    .unstack(fill_value=0)
)
comunas_x_drogas.columns = ["Nunca", "Pocas veces", "Muchas veces", "Siempre", "N/A"]
comunas_x_drogas

In [ ]:
comunas_x_trabajo = (
    casen_comunas.groupby(["COMUNA", "o15"])["expr"]
    .sum()
    .unstack(fill_value=0)
)

comunas_x_trabajo.columns = ['Patrón o empleador', 'Trabajador por cuenta propia', 'Funcionario público (Gobierno Central o Municipal)', 'Empleado u obrero de empresas públicas', 'Empreado u obrero del sector privado', 'Servicio doméstico puertas adentro', 'Servicio doméstico puertas afuera', 'FFAA y del Orden', 'Familiar no remunerado']

comunas_x_trabajo.head()

In [ ]:
trabajo_x_ingreso = (
    casen_comunas.groupby(["o15", "dautr"])["expr"].sum().unstack(fill_value=0)
)
trabajo_x_ingreso.index = [
    "Patrón o empleador",
    "Trabajador por cuenta propia",
    "Funcionario público (Gobierno Central o Municipal)",
    "Empleado u obrero de empresas públicas",
    "Empreado u obrero del sector privado",
    "Servicio doméstico puertas adentro",
    "Servicio doméstico puertas afuera",
    "FFAA y del Orden",
    "Familiar no remunerado",
]
sns.clustermap(trabajo_x_ingreso)


In [ ]:
turnout_plebiscito = pd.read_csv(AVES_ROOT / 'data' / 'external' / 'servel_2020' / 'votos_comuna_plebiscito_2020.csv', index_col=0)
turnout_plebiscito

## Posible abstención

In [ ]:
poblacion_votante_comunas = (
    casen_comunas[casen_comunas["edad"] >= 14].groupby("COMUNA")["expr"].sum())
poblacion_votante_comunas.sum()

In [ ]:
posible_abstencion = poblacion_votante_comunas - turnout.loc[
    poblacion_votante_comunas.index
].sum(axis=1)

comunas_x_abstencion = pd.get_dummies(pd.qcut(posible_abstencion, 5, labels=False))
comunas_x_abstencion.columns = ['Muy baja', 'Baja', 'Media', 'Alta', 'Muy alta']
comunas_x_abstencion

## Relación entre grillas y comunas

In [ ]:
len(square_grid)

In [ ]:
grid_intersection = gpd.overlay(square_grid[square_grid['s2_cellid'].isin(relevant_cell_ids)], comunas[['geometry', 'NOM_COMUNA']], how='intersection')
grid_intersection

In [ ]:
grid_intersection.plot(facecolor='none', edgecolor='black')

In [ ]:
grid_intersection['area_weight'] = grid_intersection.to_crs('epsg:5361').area / 1000000
grid_intersection

In [ ]:
grid_x_comuna = grid_intersection.groupby(['s2_cellid', 'NOM_COMUNA'])['area_weight'].sum().unstack(fill_value=0)
grid_x_comuna.columns = grid_x_comuna.columns.map(deaccent)
grid_x_comuna

## Atributos de grilla

In [ ]:

grid_eod = gpd.sjoin(to_point_geodataframe(recurrent_trips, 'DirCoordX', 'DirCoordY', crs='epsg:5361').to_crs('epsg:4326'), square_grid, op='within')
grid_eod

In [ ]:
grid_eod['decil_ingreso'] = pd.qcut(grid_eod['IngresoHogar'], q=10, labels=False)
grid_x_ingreso = grid_eod.groupby(['s2_cellid', 'decil_ingreso'])['FactorHogar'].sum().unstack(fill_value=0)
grid_x_ingreso

In [ ]:
grid_x_transporte = (
    grid_eod.groupby(["s2_cellid", "bin_tiempo_de_viaje"])["PesoLaboral"]
    .sum()
    .unstack()
    .pipe(normalize_rows)
)

grid_x_transporte

## Modelo de Fusión de Datos

`AVES` incorpora un _wrapper_ sobre [scikit-fusion](https://github.com/mims-harvard/scikit-fusion).

In [ ]:
list_comunas = list(set(map(deaccent, grid_intersection['NOM_COMUNA'].unique())) & set(zones['NOM_COMUNA']))
list_comunas

In [ ]:
geo_turnout_all_cells = grid_with_data.loc[relevant_cell_ids][geo_turnout.drop('s2_cellid', axis=1).columns]
geo_turnout_all_cells

EL modelo de fusión de datos necesita:

- `model_nodes`: un número de dimensiones para representar cada concepto que tenemos en los datos
- `model_relations`: las relaciones entre los conceptos, expresadas como matrices (es lo que hemos construido antes). Noten que a muchas de esas matrices le aplicamos una transformación `sqrt` para estabilizar el modelo (ya que los modelos de factorización suelen ser **lineales**).

In [ ]:
# dimensión (rank) de la representación latente de cada entidad
model_nodes = {
    "candidatos": 5,
    "comunas": 24,
    "ingreso": 4,
    "trabajo": 4,
    "narcotráfico": 2,
    "plebiscito": 2,
    "grid": 128,
    'abstencion': 2,
    'calidad_transporte': 4,
}

# relaciones entre entidades
model_relations = {
    ('candidatos', 'grid'): [geo_turnout_all_cells.T],
    ('candidatos', 'comunas'): [turnout.loc[list_comunas].T.pipe(np.sqrt)],
    ('grid', 'comunas'): [grid_x_comuna.loc[relevant_cell_ids][list_comunas]],
    ('grid', 'ingreso'): [geo_turnout_all_cells[[]].join(grid_x_ingreso).fillna(0).pipe(np.sqrt)],
    ('grid', 'calidad_transporte'): [geo_turnout_all_cells[[]].join(grid_x_transporte).fillna(0).pipe(np.sqrt)],
    ('comunas', 'plebiscito'): [turnout_plebiscito.loc[list_comunas].pipe(np.sqrt)],
    ('comunas', 'ingreso'): [comunas_x_ingreso.loc[list_comunas].pipe(np.sqrt)],
    ('comunas', 'narcotráfico'): [comunas_x_drogas.loc[list_comunas].pipe(np.sqrt)],
    ('comunas', 'calidad_transporte'): [comuna_x_transporte.loc[list_comunas].pipe(np.sqrt)],
    ('comunas', 'trabajo'): [comunas_x_trabajo.loc[list_comunas].pipe(np.sqrt)],
    ('comunas', 'abstencion'): [comunas_x_abstencion.loc[list_comunas]],
    ('trabajo', 'ingreso'): [trabajo_x_ingreso.pipe(np.sqrt)]
}

In [ ]:
seed = random.randint(0, 100000)
print(seed)

random.seed(seed)
np.random.seed(seed)

model = DataFusionModel(nodes=model_nodes, relations=model_relations)
model.fit()

## Exploración de algunos factores latentes

In [ ]:
sns.clustermap(model.factor('candidatos'))

In [ ]:
sns.clustermap(model.factor('comunas'), method='ward')

In [ ]:
sns.clustermap(model.factor('trabajo'), method='ward')

In [ ]:
sns.heatmap(model.relation_profiles('candidatos', 'abstencion')[0][1], center=0)

In [ ]:
sns.heatmap(model.relation_profiles('candidatos', 'trabajo')[0][1].pipe(np.sqrt), center=0)

## Extrapolación de datos en la grilla

El modelo extrapola atributos a la grilla con la que trabajamos. Entonces, lo usaremos para calcular tendencias de voto y luego identificar puntos clave de esas tendencias. El enfoque es aplicable a cualquier candidato, pero como ejemplo lo veremos con `GABRIEL BORIC FONT` en la elección presidencial y el voto de `APRUEBO` en el plebiscito constitucional.

In [ ]:
reconstruction = model.relation_profiles('candidatos', 'grid')[0][1].T
reconstruction

In [ ]:
grid_x_candidatos = reconstruction.pipe(standardize_columns)
grid_x_candidatos

In [ ]:
fig, axes = small_multiples_from_geodataframe(grid_with_data, 2)

palette = color_palette('magma', n_colors=4)

grid_reconstruction = square_grid[square_grid['s2_cellid'].isin(relevant_cell_ids)].join(grid_x_candidatos, on='s2_cellid')

for ax, col in zip(axes, ["GABRIEL BORIC FONT", "JOSE ANTONIO KAST RIST"]):
    choropleth_map(
        ax,
        grid_reconstruction,
        col,
        binning='fisher_jenks',
        k=5,
        #bins=bins, legend=None, palette=palette

    )
    comunas.plot(ax=ax, facecolor="none", edgecolor="#abacab", linewidth=1)
    ax.set_title(col.title())

fig.tight_layout()

In [ ]:
grid_x_tendencia = (grid_x_candidatos['GABRIEL BORIC FONT'] - grid_x_candidatos['JOSE ANTONIO KAST RIST']).rename('tendencia')
grid_x_tendencia

In [ ]:
fig, ax = small_multiples_from_geodataframe(grid_with_data, 1)

palette = color_palette('magma', n_colors=4)

grid_reconstruction = square_grid[square_grid['s2_cellid'].isin(relevant_cell_ids)].join(grid_x_tendencia, on='s2_cellid')

choropleth_map(
    ax,
    grid_reconstruction,
    'tendencia',
    binning='uniform',
    k=3,
)
comunas.plot(ax=ax, facecolor="none", edgecolor="#abacab", linewidth=1)
ax.set_title('Tendencia de votación (Boric)')

fig.tight_layout()

In [ ]:
grid_x_plebiscito_reconstruction = model.relation_profiles('grid', 'plebiscito')[0][1].pipe(standardize_columns)
grid_x_plebiscito_reconstruction

In [ ]:
fig, ax = small_multiples_from_geodataframe(grid_with_data, 1)

palette = color_palette('magma', n_colors=4)

grid_reconstruction = square_grid[square_grid['s2_cellid'].isin(relevant_cell_ids)].join(grid_x_plebiscito_reconstruction, on='s2_cellid')

choropleth_map(
    ax,
    grid_reconstruction,
    'APRUEBO',
    binning='uniform',
    k=3,
    #bins=bins, legend=None, palette=palette

)
comunas.plot(ax=ax, facecolor="none", edgecolor="#abacab", linewidth=1)
ax.set_title('Tendencia APRUEBO (Plebiscito 2021)')

fig.tight_layout()

In [ ]:
bivariate_data = (
    grid_x_plebiscito_reconstruction["APRUEBO"]
    .rename("apoyo_apruebo")
    .to_frame()
    .join(grid_x_tendencia.rename('tendencia_boric'))
)
bivariate_data


In [ ]:
bivariate_data.corr()

In [ ]:
from aves.visualization.maps.choropleth import bivariate_choropleth_map

fig, ax = small_multiples_from_geodataframe(grid_with_data, 1, height=9)

grid_reconstruction = square_grid[square_grid['s2_cellid'].isin(relevant_cell_ids)].join(bivariate_data, on='s2_cellid')

bi_color_matrix, cbar_ax = bivariate_choropleth_map(
    ax,
    grid_reconstruction,
    'apoyo_apruebo',
    'tendencia_boric',
    binning='uniform',
    k=3,
    cbar_args=dict(location='lower right', width='15%')
)
comunas.plot(ax=ax, facecolor="none", edgecolor="#abacab", linewidth=1)
ax.set_title('Tendencias bivariadas')

cbar_ax.set_xlabel('Apoyo Apruebo $\\rightarrow$', fontsize='x-small')
cbar_ax.set_ylabel('Apoyo a Boric $\\rightarrow$', fontsize='x-small')
sns.despine(ax=cbar_ax)

fig.tight_layout()